In [1]:
#Based on the PyTorch quickstart tutorial
#https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [2]:
#PyTorch primitives: torch.utils.data.Dataset stores samples and corresponding labels.
#torch.utils.data.DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets #package consists of popular datasets, model architectures, and common image transformations for computer vision.
from torchvision.transforms import ToTensor

In [3]:
#Using FashionMNIST dataset
#Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.
#Download training and test data from open datasets.

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 15658923.63it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 349538.16it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6630882.79it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5434753.84it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

